In [50]:
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import matplotlib.colors as mcolors
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import os
# from pykrige.ok import OrdinaryKriging
# import pykrige.kriging_tools as kt
# from pykrige.kriging_tools import write_asc_grid
# import gstools as gs

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import Rbf, RBFInterpolator
from gstools import Gaussian, krige
import gstools as gs
from matplotlib import cm

In [51]:
data_file = "../../../data/one_day_data/2020/08/2020-08-01"
df = pd.read_csv(data_file + '/4-30.csv', index_col=0)
df.head()

,LON,LAT,hour-rain,AT1,RH1,SOL,WD1,WS1,PRS,SLP
Vlz-Bagbaguin_00174731,121.0009,14.7134,0.0,28.200000,53.200000,27.000000,226.000000,2.000000,1007.600000,1010.500000
NagNayon_00181273,121.0285,14.7192,0.0,29.400000,54.600000,54.000000,173.000000,0.600000,1004.800000,1011.100000
Sinegue_00181300,120.9319,14.4599,39.0,24.800000,81.500000,41.000000,89.000000,0.200000,1008.900000,1010.600000
DLSAU_00174722,120.9974,14.6705,0.0,26.532066,73.482975,178.104132,148.930579,0.326116,1008.667273,1011.166942
ASTI_00173457,121.0721,14.6471,0.0,28.600000,51.400000,219.000000,320.000000,3.200000,1002.800000,1010.500000


In [67]:
def position_target(df):
    x, y = df['LON'].values.tolist(), df['LAT'].values.tolist()
    
    cond_pos = [x, y]
    
    val = df['hour-rain'].values
    
    for i in range(len(cond_pos)):
        x, y = cond_pos[0][i], cond_pos[1][i]
        tmp = df.loc[df['LON'] == x]
        target = tmp['hour-rain'].values[0]
        assert(target == val[i])
        
    return cond_pos, val
    

def krig_model(dim=2, var=0.5, len_scale=2):
    model = Gaussian(dim=dim, var=var, len_scale=len_scale)
    
    cond_pos, target = position_target(df)
    
    kr1 = krige.Simple(model=model, mean=1, cond_pos=cond_pos, cond_val=target)
    kr2 = krige.Ordinary(model=model, cond_pos=cond_pos, cond_val=target)
    
    grid_lon = np.round(np.linspace(120.90, 121.150, 50), decimals=3)
    grid_lat = np.round(np.linspace(14.350, 14.760, 50), decimals=3)
    xi, yi = np.meshgrid(grid_lon, grid_lat)
    xgrid = np.around(np.mgrid[120.90:121.150:50j, 14.350:14.760:50j], decimals=3)
    xfloat = np.around(xgrid, decimals=3).reshape(2, -1).T
    
    kr1(xfloat)
    kr2(xfloat)
    
    print(kr1.field)
    print(kr2.field)
    

In [68]:
krig_model()

[1. 1. 1. ... 1. 1. 1.]
[-252736.88670796 -252736.88670796 -252736.88670796 ... -252736.88670796
 -252736.88670796 -252736.88670796]
